# Controllable Music Recommender

This notebook builds a flexible, controllable music recommendation system that combines:
- **Track2Vec embeddings** from notebook 02 (semantic similarity)
- **Artist metadata** from notebook 03 (genres, popularity, followers)

## Control Knobs

The recommender allows you to tune:
1. **Popularity** - From obscure underground tracks to mainstream hits
2. **Artist Diversity** - Discover new artists vs. stick to familiar ones
3. **Genre Focus** - Filter or boost specific genres
4. **Exploration** - Random discovery vs. safe similarity-based picks
5. **Freshness** - Exclude artists you already know

In [23]:
import pandas as pd
import numpy as np
from pathlib import Path
from gensim.models import KeyedVectors
from collections import defaultdict, Counter
from typing import List, Optional, Dict, Set, Tuple
import warnings
import json
warnings.filterwarnings('ignore')

# Paths
MODEL_DIR = Path("../models")
DATA_DIR = Path("../data/processed")

## Load Models and Data

In [24]:
# Load track embeddings
wv = KeyedVectors.load(str(MODEL_DIR / "track2vec.wordvectors"))
print(f"Loaded {len(wv):,} track embeddings (dim={wv.vector_size})")

# Load track metadata
track_df = pd.read_parquet(MODEL_DIR / "track_metadata.parquet")
print(f"Loaded {len(track_df):,} tracks metadata")

# Load artist info
artist_df = pd.read_parquet(DATA_DIR / "artist_info.parquet")
print(f"Loaded {len(artist_df):,} artists info")

Loaded 62,476 track embeddings (dim=128)
Loaded 252,999 tracks metadata
Loaded 49,124 artists info


In [25]:
# Build enriched dataset (merge track metadata with artist info)
def extract_id(uri):
    return uri.split(":")[-1]

track_df['track_id'] = track_df['track_uri'].apply(extract_id)
track_df['artist_id'] = track_df['artist_uri'].apply(extract_id)

# Load playlist counts from raw data
PLAYLIST_DATA_PATH = Path.home() / ".cache/kagglehub/datasets/himanshuwagh/spotify-million/versions/1/data"
slice_files = sorted(PLAYLIST_DATA_PATH.glob("mpd.slice.*.json"))

track_playlist_count = Counter()
for slice_file in slice_files:
    with open(slice_file) as f:
        data = json.load(f)
        for playlist in data["playlists"]:
            for track in playlist["tracks"]:
                track_playlist_count[track["track_uri"]] += 1

track_df['playlist_count'] = track_df['track_uri'].map(track_playlist_count).fillna(0).astype(int)

# Merge with artist info
artist_df_slim = artist_df.drop(columns=['artist_name'])
enriched_df = track_df.merge(artist_df_slim, on='artist_id', how='left')

# Fill missing values
# Convert genres: handle numpy arrays, lists, and NaN values
def normalize_genres(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return []
    if hasattr(x, 'tolist'):  # numpy array
        return x.tolist()
    if isinstance(x, list):
        return x
    return []

enriched_df['genres'] = enriched_df['genres'].apply(normalize_genres)
enriched_df['artist_popularity'] = enriched_df['artist_popularity'].fillna(50)
enriched_df['artist_followers'] = enriched_df['artist_followers'].fillna(0)

# Filter to only tracks in vocabulary
vocab_set = set(wv.key_to_index.keys())
enriched_df['in_vocab'] = enriched_df['track_uri'].isin(vocab_set)
print(f"\nEnriched dataset: {len(enriched_df):,} tracks")
print(f"Tracks in embedding vocabulary: {enriched_df['in_vocab'].sum():,}")


Enriched dataset: 257,667 tracks
Tracks in embedding vocabulary: 63,555


In [26]:
# Create fast lookup dictionaries
# Drop duplicates first (keep first occurrence) to ensure unique index
enriched_unique = enriched_df.drop_duplicates(subset='track_uri', keep='first')
print(f"Unique tracks after dedup: {len(enriched_unique):,} (dropped {len(enriched_df) - len(enriched_unique):,} duplicates)")

uri_to_info = enriched_unique.set_index('track_uri').to_dict('index')
artist_to_tracks = defaultdict(list)
genre_to_tracks = defaultdict(list)

for uri, info in uri_to_info.items():
    if uri in vocab_set:
        artist_to_tracks[info['artist_name']].append(uri)
        for genre in info['genres']:
            genre_to_tracks[genre].append(uri)

print(f"Artists with tracks in vocab: {len(artist_to_tracks):,}")
print(f"Unique genres: {len(genre_to_tracks):,}")

Unique tracks after dedup: 252,999 (dropped 4,668 duplicates)
Artists with tracks in vocab: 12,714
Unique genres: 653


In [27]:
# DEBUG: Check if genres are being loaded properly
print("Checking genre data...")
print(f"\n1. Artist DF genres column sample:")
print(artist_df['genres'].head(10).tolist())

print(f"\n2. Enriched DF genres column sample:")
print(enriched_df['genres'].head(10).tolist())

print(f"\n3. Non-empty genres in enriched_df: {(enriched_df['genres'].apply(len) > 0).sum():,}")

# Check a sample from uri_to_info
sample_uri = list(uri_to_info.keys())[0]
print(f"\n4. Sample uri_to_info entry genres: {uri_to_info[sample_uri].get('genres', 'NOT FOUND')}")

Checking genre data...

1. Artist DF genres column sample:
[array(['hip hop'], dtype=object), array(['pop'], dtype=object), array([], dtype=object), array([], dtype=object), array(['reggae'], dtype=object), array(['r&b'], dtype=object), array([], dtype=object), array([], dtype=object), array(['southern hip hop', 'hip hop'], dtype=object), array([], dtype=object)]

2. Enriched DF genres column sample:
[['hip hop'], ['pop'], [], [], ['reggae'], ['r&b'], ['r&b'], [], [], ['southern hip hop', 'hip hop']]

3. Non-empty genres in enriched_df: 175,592

4. Sample uri_to_info entry genres: ['hip hop']


## Explore Available Genres

In [28]:
# Show top genres by track count
genre_counts = {g: len(tracks) for g, tracks in genre_to_tracks.items()}
top_genres = sorted(genre_counts.items(), key=lambda x: -x[1])[:40]

print("Top 40 genres by track count:")
print("=" * 50)
for i, (genre, count) in enumerate(top_genres):
    print(f"{i+1:2d}. {genre:30s} {count:5,d} tracks")

Top 40 genres by track count:
 1. country                        3,805 tracks
 2. christian                      2,398 tracks
 3. worship                        2,210 tracks
 4. ccm                            1,726 tracks
 5. rock                           1,481 tracks
 6. edm                            1,481 tracks
 7. rap                            1,480 tracks
 8. emo                            1,319 tracks
 9. hip hop                        1,247 tracks
10. pop punk                       1,235 tracks
11. r&b                            1,214 tracks
12. acoustic country               1,146 tracks
13. classic rock                   1,138 tracks
14. christmas                      1,010 tracks
15. classic country                  986 tracks
16. old school hip hop               958 tracks
17. metal                            929 tracks
18. pop worship                      914 tracks
19. indie folk                       855 tracks
20. christian alternative rock       832 tracks
21. christ

## The Controllable Recommender Class

In [ ]:
class ControllableRecommender:
    """
    A music recommender with adjustable control knobs.
    
    Control Parameters:
    - popularity: 0.0 (obscure) to 1.0 (mainstream)
    - artist_diversity: 0.0 (same artists) to 1.0 (new artists only)
    - exploration: 0.0 (safe/similar) to 1.0 (random/adventurous)
    - genres: list of genres to include (None = all)
    - exclude_genres: list of genres to exclude
    - exclude_artists: set of artist names to exclude
    """
    
    def __init__(self, embeddings: KeyedVectors, track_info: Dict, 
                 artist_tracks: Dict, genre_tracks: Dict):
        self.wv = embeddings
        self.track_info = track_info
        self.artist_tracks = artist_tracks
        self.genre_tracks = genre_tracks
        self.vocab = set(embeddings.key_to_index.keys())
        
        # Precompute popularity percentiles for faster lookup
        playlist_counts = [info.get('playlist_count', 0) for info in track_info.values()]
        self.popularity_percentiles = np.percentile(playlist_counts, np.arange(0, 101, 1))
        
        artist_pops = [info.get('artist_popularity', 50) for info in track_info.values()]
        self.artist_pop_percentiles = np.percentile(artist_pops, np.arange(0, 101, 1))
    
    def _get_track_display(self, uri: str) -> str:
        """Format track for display."""
        info = self.track_info.get(uri, {})
        return f"{info.get('track_name', 'Unknown')} - {info.get('artist_name', 'Unknown')}"
    
    def _compute_popularity_score(self, uri: str) -> float:
        """Compute normalized popularity score (0-1) for a track."""
        info = self.track_info.get(uri, {})
        playlist_count = info.get('playlist_count', 0)
        artist_pop = info.get('artist_popularity', 50)
        
        # Combine playlist count and artist popularity
        playlist_percentile = np.searchsorted(self.popularity_percentiles, playlist_count) / 100
        artist_percentile = artist_pop / 100
        
        return 0.6 * playlist_percentile + 0.4 * artist_percentile
    
    def _filter_candidates(self, candidates: List[Tuple[str, float]],
                           seed_artists: Set[str],
                           genres: Optional[List[str]] = None,
                           exclude_genres: Optional[List[str]] = None,
                           exclude_artists: Optional[Set[str]] = None,
                           artist_diversity: float = 0.5) -> List[Tuple[str, float]]:
        """Filter candidates based on constraints."""
        filtered = []
        exclude_artists = exclude_artists or set()
        exclude_genres = set(exclude_genres) if exclude_genres else set()
        include_genres = set(genres) if genres else None
        
        for uri, score in candidates:
            info = self.track_info.get(uri, {})
            artist = info.get('artist_name', '')
            track_genres = set(info.get('genres', []))
            
            # Exclude specific artists
            if artist in exclude_artists:
                continue
            
            # Genre filtering
            if exclude_genres and track_genres & exclude_genres:
                continue
            if include_genres and not (track_genres & include_genres):
                continue
            
            # Artist diversity penalty
            if artist in seed_artists and artist_diversity > 0:
                # Reduce score for same-artist tracks based on diversity setting
                score = score * (1 - artist_diversity * 0.8)
            
            filtered.append((uri, score))
        
        return filtered
    
    def _apply_popularity_weighting(self, candidates: List[Tuple[str, float]],
                                     popularity: float) -> List[Tuple[str, float]]:
        """
        Adjust scores based on popularity preference.
        popularity = 0.0 -> prefer obscure
        popularity = 0.5 -> neutral
        popularity = 1.0 -> prefer mainstream
        """
        if popularity == 0.5:
            return candidates
        
        weighted = []
        for uri, sim_score in candidates:
            pop_score = self._compute_popularity_score(uri)
            
            if popularity > 0.5:
                # Boost popular tracks
                weight = 1 + (popularity - 0.5) * 2 * pop_score
            else:
                # Boost obscure tracks (invert popularity)
                weight = 1 + (0.5 - popularity) * 2 * (1 - pop_score)
            
            weighted.append((uri, sim_score * weight))
        
        return weighted
    
    def _apply_exploration(self, candidates: List[Tuple[str, float]],
                           exploration: float, n: int) -> List[str]:
        """
        Select final tracks with exploration factor.
        exploration = 0.0 -> always pick top-scoring
        exploration = 1.0 -> random sampling weighted by score
        """
        if not candidates:
            return []
        
        if exploration == 0.0:
            # Deterministic: top N by score
            sorted_candidates = sorted(candidates, key=lambda x: -x[1])
            return [uri for uri, _ in sorted_candidates[:n]]
        
        # Probabilistic sampling
        uris = [c[0] for c in candidates]
        scores = np.array([c[1] for c in candidates])
        
        # Temperature-based softmax
        temperature = 0.1 + exploration * 2.0  # Range: 0.1 (sharp) to 2.1 (flat)
        exp_scores = np.exp(scores / temperature)
        probs = exp_scores / exp_scores.sum()
        
        # Sample without replacement
        n_sample = min(n, len(uris))
        selected_indices = np.random.choice(len(uris), size=n_sample, replace=False, p=probs)
        
        return [uris[i] for i in selected_indices]
    
    def recommend(self,
                  seed_tracks: List[str],
                  n: int = 10,
                  popularity: float = 0.5,
                  artist_diversity: float = 0.5,
                  exploration: float = 0.0,
                  genres: Optional[List[str]] = None,
                  exclude_genres: Optional[List[str]] = None,
                  exclude_artists: Optional[Set[str]] = None,
                  exclude_seeds: bool = True,
                  candidate_pool_size: int = 500) -> pd.DataFrame:
        """
        Generate recommendations with controllable parameters.
        
        Args:
            seed_tracks: List of track URIs to base recommendations on
            n: Number of recommendations to return
            popularity: 0.0 (obscure) to 1.0 (mainstream)
            artist_diversity: 0.0 (same artists OK) to 1.0 (new artists only)
            exploration: 0.0 (deterministic) to 1.0 (random sampling)
            genres: List of genres to include (None = all)
            exclude_genres: List of genres to exclude
            exclude_artists: Set of artist names to exclude
            exclude_seeds: Whether to exclude seed tracks from results
            candidate_pool_size: Size of initial candidate pool
        
        Returns:
            DataFrame with recommendations
        """
        # Validate seeds
        valid_seeds = [t for t in seed_tracks if t in self.vocab]
        if not valid_seeds:
            print("No valid seed tracks found in vocabulary!")
            print(f"  Provided {len(seed_tracks)} seeds, but none are in the {len(self.vocab):,} track vocabulary.")
            print(f"  Seed URIs: {seed_tracks[:3]}...")
            return pd.DataFrame()
        
        # Get seed artists
        seed_artists = {self.track_info.get(uri, {}).get('artist_name', '') for uri in valid_seeds}
        seed_set = set(seed_tracks) if exclude_seeds else set()
        
        # Compute centroid of seed tracks
        seed_vectors = [self.wv[t] for t in valid_seeds]
        centroid = np.mean(seed_vectors, axis=0)
        
        # Get large candidate pool
        similar = self.wv.similar_by_vector(centroid, topn=candidate_pool_size)
        candidates = [(uri, score) for uri, score in similar if uri not in seed_set]
        
        # DEBUG: Check pipeline stages
        candidates_before_filter = len(candidates)
        
        # Apply filters
        candidates = self._filter_candidates(
            candidates, seed_artists, genres, exclude_genres, exclude_artists, artist_diversity
        )
        
        candidates_after_filter = len(candidates)
        
        # Apply popularity weighting
        candidates = self._apply_popularity_weighting(candidates, popularity)
        
        # Apply exploration and select final tracks
        selected_uris = self._apply_exploration(candidates, exploration, n)
        
        # DEBUG output if empty results
        if not selected_uris:
            print(f"DEBUG: {len(valid_seeds)} valid seeds, {candidates_before_filter} candidates found, {candidates_after_filter} after filtering")
        
        # Build results DataFrame
        results = []
        for uri in selected_uris:
            info = self.track_info.get(uri, {})
            results.append({
                'track': self._get_track_display(uri),
                'artist': info.get('artist_name', 'Unknown'),
                'genres': ', '.join(info.get('genres', [])[:3]),
                'popularity': int(info.get('artist_popularity', 0)),
                'playlist_count': info.get('playlist_count', 0),
                'track_uri': uri
            })
        
        return pd.DataFrame(results)
    
    def search_tracks(self, query: str, limit: int = 20) -> pd.DataFrame:
        """Search for tracks by name or artist."""
        query = query.lower()
        matches = []
        
        for uri, info in self.track_info.items():
            if query in info.get('track_name', '').lower() or query in info.get('artist_name', '').lower():
                playlist_count = info.get('playlist_count', 0)
                artist_pop = info.get('artist_popularity', 0)
                
                # Calculate percentiles
                track_pop_pct = int(np.searchsorted(self.popularity_percentiles, playlist_count))
                artist_pop_pct = int(np.searchsorted(self.artist_pop_percentiles, artist_pop))
                
                matches.append({
                    'track': self._get_track_display(uri),
                    'artist': info.get('artist_name', 'Unknown'),
                    'genres': ', '.join(info.get('genres', [])[:3]),
                    'playlist_count': playlist_count,
                    'track_pop_pct': track_pop_pct,
                    'artist_popularity': int(artist_pop),
                    'artist_pop_pct': artist_pop_pct,
                    'in_vocab': uri in self.vocab,
                    'track_uri': uri
                })
                if len(matches) >= limit:
                    break
        
        return pd.DataFrame(matches)
    
    def get_genre_tracks(self, genre: str, limit: int = 20) -> pd.DataFrame:
        """Get sample tracks from a specific genre."""
        uris = self.genre_tracks.get(genre, [])
        
        results = []
        for uri in uris[:limit]:
            info = self.track_info.get(uri, {})
            results.append({
                'track': self._get_track_display(uri),
                'artist': info.get('artist_name', 'Unknown'),
                'popularity': int(info.get('artist_popularity', 0)),
                'track_uri': uri
            })
        
        return pd.DataFrame(results)

In [30]:
# Initialize the recommender
recommender = ControllableRecommender(
    embeddings=wv,
    track_info=uri_to_info,
    artist_tracks=artist_to_tracks,
    genre_tracks=genre_to_tracks
)

print("Recommender initialized!")

Recommender initialized!


## Demo: Search for Seed Tracks

In [31]:
# Search for tracks to use as seeds
print("Searching for 'Kendrick Lamar'...")
recommender.search_tracks("kendrick lamar", limit=10)

Searching for 'Kendrick Lamar'...


,track,artist,genres,playlist_count,track_pop_pct,artist_popularity,artist_pop_pct,in_vocab,track_uri
0,m.A.A.d city - Kendrick Lamar,Kendrick Lamar,"hip hop, west coast hip hop",380,100,91,100,True,spotify:track:439TlnnznSiBbQbgXiBqAd
1,DNA. - Kendrick Lamar,Kendrick Lamar,"hip hop, west coast hip hop",521,100,91,100,True,spotify:track:6HZILIRieu8S0iqY8kIKhj
2,HUMBLE. - Kendrick Lamar,Kendrick Lamar,"hip hop, west coast hip hop",875,100,91,100,True,spotify:track:7KXjTSCq5nL1LoYtL7XAwS
3,GOD. - Kendrick Lamar,Kendrick Lamar,"hip hop, west coast hip hop",66,99,91,100,True,spotify:track:5p3NP9mQblDsVJLdfNZJ3I
4,Don't Wanna Know (feat. Kendrick Lamar) - Maro...,Maroon 5,pop,261,100,88,99,True,spotify:track:2QbFClFyhMMtiurUjuQlAe
5,Swimming Pools (Drank) - Kendrick Lamar,Kendrick Lamar,"hip hop, west coast hip hop",336,100,91,100,True,spotify:track:6REbwUNlppTfcnV4d4ZoZi
6,King Kunta - Kendrick Lamar,Kendrick Lamar,"hip hop, west coast hip hop",282,100,91,100,True,spotify:track:0N3W5peJUQtI4eyR6GJT5O
7,LOYALTY. FEAT. RIHANNA. - Kendrick Lamar,Kendrick Lamar,"hip hop, west coast hip hop",267,100,91,100,True,spotify:track:6SwRhMLwNqEi6alNPVG00n
8,LOVE. FEAT. ZACARI. - Kendrick Lamar,Kendrick Lamar,"hip hop, west coast hip hop",330,100,91,100,True,spotify:track:6PGoSes0D9eUDeeAafB2As
9,Swimming Pools (Drank) - Extended Version - Ke...,Kendrick Lamar,"hip hop, west coast hip hop",145,100,91,100,True,spotify:track:5ujh1I7NZH5agbwf7Hp8Hc


## Demo: Basic Recommendations

In [32]:
# Find some seed tracks
drake_results = recommender.search_tracks("drake")
drake_uris = drake_results[drake_results['in_vocab']]['track_uri'].tolist()[:3]

print("Seed tracks:")
for uri in drake_uris:
    print(f"  - {recommender._get_track_display(uri)}")

print("\n" + "="*60)
print("Default recommendations (balanced settings):")
print("="*60)
recommender.recommend(drake_uris, n=10)

Seed tracks:
  - Portland - Drake
  - Say Something (Featuring Drake) - Timbaland
  - Preach - Drake

Default recommendations (balanced settings):


,track,artist,genres,popularity,playlist_count,track_uri
0,Hoe On A Mission - Migos,Migos,trap,77,3,spotify:track:4dvejkvTqHa8zjHG8mwBSh
1,Wet Dreamz - Live - J. Cole,J. Cole,rap,85,5,spotify:track:4VUpgvJxbriXHrMrsb60HB
2,My Story (Ten Toes Down Challenge) - T the Mes...,T the Messenger,,5,3,spotify:track:49jPkK44gc0y92xc2aP4OC
3,Oh Hi Hater (Hiatus) (feat. Fortune) - Mike Wi...,Mike WiLL Made-It,,66,3,spotify:track:6hIMVLEscwFzVdnh344tUD
4,Back On The Bullshit - 2 Chainz,2 Chainz,"southern hip hop, rap",76,6,spotify:track:6IkWEB1thSgymYQJXFqJJO
5,The Truth - Mark Battles,Mark Battles,,42,3,spotify:track:5unUu3S6XjuNH9A1Mj9K6d
6,For Ya - Bandana Split,Bandana Split,,8,4,spotify:track:3pnwRURls7p0waDfWYKsFx
7,Ball Drop - Fabolous,Fabolous,,68,4,spotify:track:0NURTfgmv3sUw5r13TQsgg
8,Missing You - YFN Lucci,YFN Lucci,,66,3,spotify:track:418YSHZlCzQAmsffD2ding
9,Snake In The Grass - feat. Cartier Kitten [Exp...,Waka Flocka Flame,,72,4,spotify:track:0rkAUNogg99pvVPs4xqN2h


## Control Knob Examples

### 1. Popularity Control: Mainstream vs. Underground

In [33]:
print("=" * 60)
print("MAINSTREAM MODE (popularity=1.0)")
print("Recommending the most popular similar tracks")
print("=" * 60)
display(recommender.recommend(drake_uris, n=10, popularity=1.0))

print("\n" + "=" * 60)
print("UNDERGROUND MODE (popularity=0.0)")
print("Recommending obscure, lesser-known tracks")
print("=" * 60)
display(recommender.recommend(drake_uris, n=10, popularity=0.0))

MAINSTREAM MODE (popularity=1.0)
Recommending the most popular similar tracks


,track,artist,genres,popularity,playlist_count,track_uri
0,Wet Dreamz - Live - J. Cole,J. Cole,rap,85,5,spotify:track:4VUpgvJxbriXHrMrsb60HB
1,March Madness - Future,Future,rap,91,58,spotify:track:6hVGw7yVUd2bpdHMPCWrtN
2,Real Nigga - 21 Savage,21 Savage,,88,7,spotify:track:5V57eUQoTrMCZqonhzi8B9
3,Dome (feat. Duke) - Young Thug,Young Thug,"rap, melodic rap, trap",85,9,spotify:track:5kBmoWvzfQgpd1pVGwFwab
4,Everything 1k - Kodak Black,Kodak Black,,82,10,spotify:track:78AucGxdmVa4Bx6HwDPOS4
5,When I Was Broke - Future,Future,rap,91,6,spotify:track:7Co6yOkvI0GvijiYcMuT84
6,Kno The Meaning - Future,Future,rap,91,22,spotify:track:2depzNvnAgMFKYnm3moXIc
7,My Collection - Future,Future,rap,91,16,spotify:track:0526DaOuV7mRlPKxhMSJKw
8,Back On The Bullshit - 2 Chainz,2 Chainz,"southern hip hop, rap",76,6,spotify:track:6IkWEB1thSgymYQJXFqJJO
9,March Madness - Future,Future,rap,91,73,spotify:track:26oF6WjkOjDIRK9YsdZp2l



UNDERGROUND MODE (popularity=0.0)
Recommending obscure, lesser-known tracks


,track,artist,genres,popularity,playlist_count,track_uri
0,My Story (Ten Toes Down Challenge) - T the Mes...,T the Messenger,,5,3,spotify:track:49jPkK44gc0y92xc2aP4OC
1,Bounce - Ready Wave,Ready Wave,,0,3,spotify:track:6VzxGogNkfI8PVkazYA2eG
2,Im Selfish - Silky P,Silky P,,0,3,spotify:track:4qRBCEXmzBodcVDkNE4Cop
3,SheDGAF - Broderick Batts,Broderick Batts,,0,4,spotify:track:72tVfRP3l8Oi0Cf6wF1ejm
4,Spaceships - Cyph,Cyph,,5,3,spotify:track:7GfTwSNozRJQbt5PkyutBx
5,For Ya - Bandana Split,Bandana Split,,8,4,spotify:track:3pnwRURls7p0waDfWYKsFx
6,Kodak Black Lil Uzi Vert Lil Yachty Cypher - X...,XXL Freshman,,0,3,spotify:track:01pw3e2s4PExku8cLYylH4
7,One Night - Extended Version - Yung Boat,Yung Boat,,0,3,spotify:track:0bUIGNaihS7O21CaibZVcL
8,Wicked - Purple Reign,Purple Reign,philly soul,0,4,spotify:track:0bVPZsKQtkBeZCWwN81rT2
9,Mrs Mary Mack - DJ HAT,DJ HAT,,0,3,spotify:track:5iEsDcpyWpSKzj5R19NQAd


### 2. Artist Diversity: New Artists vs. Familiar

In [34]:
print("=" * 60)
print("SAME ARTISTS OK (artist_diversity=0.0)")
print("May include more tracks from the seed artists")
print("=" * 60)
display(recommender.recommend(drake_uris, n=10, artist_diversity=0.0))

print("\n" + "=" * 60)
print("NEW ARTISTS ONLY (artist_diversity=1.0)")
print("Prioritize discovering new artists")
print("=" * 60)
display(recommender.recommend(drake_uris, n=10, artist_diversity=1.0))

SAME ARTISTS OK (artist_diversity=0.0)
May include more tracks from the seed artists


,track,artist,genres,popularity,playlist_count,track_uri
0,Can't Have Everything - Drake,Drake,rap,98,44,spotify:track:5f5r2N4Lp9WoULWPH9zp2W
1,Hoe On A Mission - Migos,Migos,trap,77,3,spotify:track:4dvejkvTqHa8zjHG8mwBSh
2,No Long Talk - Drake,Drake,rap,98,71,spotify:track:6CfrYuD3YRDYdYvH9jNtXY
3,Wet Dreamz - Live - J. Cole,J. Cole,rap,85,5,spotify:track:4VUpgvJxbriXHrMrsb60HB
4,My Story (Ten Toes Down Challenge) - T the Mes...,T the Messenger,,5,3,spotify:track:49jPkK44gc0y92xc2aP4OC
5,Oh Hi Hater (Hiatus) (feat. Fortune) - Mike Wi...,Mike WiLL Made-It,,66,3,spotify:track:6hIMVLEscwFzVdnh344tUD
6,Back On The Bullshit - 2 Chainz,2 Chainz,"southern hip hop, rap",76,6,spotify:track:6IkWEB1thSgymYQJXFqJJO
7,The Truth - Mark Battles,Mark Battles,,42,3,spotify:track:5unUu3S6XjuNH9A1Mj9K6d
8,For Ya - Bandana Split,Bandana Split,,8,4,spotify:track:3pnwRURls7p0waDfWYKsFx
9,Ball Drop - Fabolous,Fabolous,,68,4,spotify:track:0NURTfgmv3sUw5r13TQsgg



NEW ARTISTS ONLY (artist_diversity=1.0)
Prioritize discovering new artists


,track,artist,genres,popularity,playlist_count,track_uri
0,Hoe On A Mission - Migos,Migos,trap,77,3,spotify:track:4dvejkvTqHa8zjHG8mwBSh
1,Wet Dreamz - Live - J. Cole,J. Cole,rap,85,5,spotify:track:4VUpgvJxbriXHrMrsb60HB
2,My Story (Ten Toes Down Challenge) - T the Mes...,T the Messenger,,5,3,spotify:track:49jPkK44gc0y92xc2aP4OC
3,Oh Hi Hater (Hiatus) (feat. Fortune) - Mike Wi...,Mike WiLL Made-It,,66,3,spotify:track:6hIMVLEscwFzVdnh344tUD
4,Back On The Bullshit - 2 Chainz,2 Chainz,"southern hip hop, rap",76,6,spotify:track:6IkWEB1thSgymYQJXFqJJO
5,The Truth - Mark Battles,Mark Battles,,42,3,spotify:track:5unUu3S6XjuNH9A1Mj9K6d
6,For Ya - Bandana Split,Bandana Split,,8,4,spotify:track:3pnwRURls7p0waDfWYKsFx
7,Ball Drop - Fabolous,Fabolous,,68,4,spotify:track:0NURTfgmv3sUw5r13TQsgg
8,Missing You - YFN Lucci,YFN Lucci,,66,3,spotify:track:418YSHZlCzQAmsffD2ding
9,Snake In The Grass - feat. Cartier Kitten [Exp...,Waka Flocka Flame,,72,4,spotify:track:0rkAUNogg99pvVPs4xqN2h


### 3. Exploration: Safe vs. Adventurous

In [35]:
print("=" * 60)
print("SAFE MODE (exploration=0.0)")
print("Always picks the most similar tracks (deterministic)")
print("=" * 60)
display(recommender.recommend(drake_uris, n=10, exploration=0.0))

print("\n" + "=" * 60)
print("ADVENTUROUS MODE (exploration=0.8)")
print("Randomly samples from good candidates (run multiple times!)")
print("=" * 60)
display(recommender.recommend(drake_uris, n=10, exploration=0.8))

SAFE MODE (exploration=0.0)
Always picks the most similar tracks (deterministic)


,track,artist,genres,popularity,playlist_count,track_uri
0,Hoe On A Mission - Migos,Migos,trap,77,3,spotify:track:4dvejkvTqHa8zjHG8mwBSh
1,Wet Dreamz - Live - J. Cole,J. Cole,rap,85,5,spotify:track:4VUpgvJxbriXHrMrsb60HB
2,My Story (Ten Toes Down Challenge) - T the Mes...,T the Messenger,,5,3,spotify:track:49jPkK44gc0y92xc2aP4OC
3,Oh Hi Hater (Hiatus) (feat. Fortune) - Mike Wi...,Mike WiLL Made-It,,66,3,spotify:track:6hIMVLEscwFzVdnh344tUD
4,Back On The Bullshit - 2 Chainz,2 Chainz,"southern hip hop, rap",76,6,spotify:track:6IkWEB1thSgymYQJXFqJJO
5,The Truth - Mark Battles,Mark Battles,,42,3,spotify:track:5unUu3S6XjuNH9A1Mj9K6d
6,For Ya - Bandana Split,Bandana Split,,8,4,spotify:track:3pnwRURls7p0waDfWYKsFx
7,Ball Drop - Fabolous,Fabolous,,68,4,spotify:track:0NURTfgmv3sUw5r13TQsgg
8,Missing You - YFN Lucci,YFN Lucci,,66,3,spotify:track:418YSHZlCzQAmsffD2ding
9,Snake In The Grass - feat. Cartier Kitten [Exp...,Waka Flocka Flame,,72,4,spotify:track:0rkAUNogg99pvVPs4xqN2h



ADVENTUROUS MODE (exploration=0.8)
Randomly samples from good candidates (run multiple times!)


,track,artist,genres,popularity,playlist_count,track_uri
0,No Peace - Amir Obe,Amir Obe,,46,3,spotify:track:1JEQgcWOCGCl0e07PCeybs
1,The Meaning - FKi 1st,FKi 1st,,38,11,spotify:track:6BC5qv70s01qQpFyKSsuLV
2,New Level REMIX - A$AP Ferg,A$AP Ferg,rap,70,11,spotify:track:36evXa8bNvfBYNzePEWhG1
3,Play Wit Yo' Bitch - Young Dolph,Young Dolph,"southern hip hop, trap",74,10,spotify:track:1QstbjmlSF0xMxGIdvhiGh
4,Sloppy Toppy - Migos,Migos,trap,77,16,spotify:track:5ftHKQmo0AyR4VcLEEjl4T
5,One Night - Extended Version - Yung Boat,Yung Boat,,0,3,spotify:track:0bUIGNaihS7O21CaibZVcL
6,Keep Up - 070,070,,16,3,spotify:track:21bjA5FwtW90Wdtm5ygsMg
7,Die This Way (feat. Matt Black & Joey Tee) - H...,Hopsin,,58,3,spotify:track:0BUAsT7wqcApoN4PN1uNwG
8,Cocaine Castle - A$AP Ferg,A$AP Ferg,rap,70,6,spotify:track:4q1ThGh6IpWSd9ixvGfuHi
9,Miami - DeJ Loaf,DeJ Loaf,,66,3,spotify:track:25I7vbfGyfkNE03EE7Dwwq


### 4. Genre Filtering

In [36]:
# Get some rock seeds
rock_results = recommender.search_tracks("nirvana")
rock_uris = rock_results[rock_results['in_vocab']]['track_uri'].tolist()[:3]

print("Seed tracks:")
for uri in rock_uris:
    print(f"  - {recommender._get_track_display(uri)}")

print("\n" + "=" * 60)
print("ROCK & ALTERNATIVE ONLY")
print("=" * 60)
display(recommender.recommend(
    rock_uris, n=10, 
    genres=['rock', 'alternative rock', 'grunge', 'indie rock']
))

print("\n" + "=" * 60)
print("EXCLUDE ROCK - FIND SIMILAR VIBES IN OTHER GENRES")
print("=" * 60)
display(recommender.recommend(
    rock_uris, n=10,
    exclude_genres=['rock', 'alternative rock', 'grunge', 'hard rock']
))

Seed tracks:
  - Heart-Shaped Box - Nirvana
  - All Apologies - Nirvana
  - Smells Like Teen Spirit - Nirvana

ROCK & ALTERNATIVE ONLY


,track,artist,genres,popularity,playlist_count,track_uri
0,Big Me - Foo Fighters,Foo Fighters,"rock, post-grunge, alternative rock",78,5,spotify:track:6pb5BBnIM5IM7R1cqag6rE
1,Plush - Stone Temple Pilots,Stone Temple Pilots,"grunge, alternative rock, post-grunge",71,71,spotify:track:3jJZVeExYzVYiV6Y9Fl3DX
2,By The Way - Red Hot Chili Peppers,Red Hot Chili Peppers,"funk rock, alternative rock, rock",85,3,spotify:track:5pAGl4nTCwcXdce769jJuI
3,Cherub Rock - The Smashing Pumpkins,The Smashing Pumpkins,"alternative rock, rock",76,35,spotify:track:0fIC59ZyS8ThtUF0Um6zuD
4,Creep - Stone Temple Pilots,Stone Temple Pilots,"grunge, alternative rock, post-grunge",71,38,spotify:track:0elH6ukMGLyCHJavHutEza
5,Today - The Smashing Pumpkins,The Smashing Pumpkins,"alternative rock, rock",76,22,spotify:track:4XizBlyqR7ZGVTX0Fyonm2
6,Tomorrow - Remastered - Silverchair,Silverchair,"grunge, post-grunge",60,12,spotify:track:24tO365YW6lcZr1hN4Ukzj
7,Jeremy - Pearl Jam,Pearl Jam,"grunge, rock, post-grunge",78,54,spotify:track:62nQ8UZVqR2RMvkJHkcO2o
8,Epic - Faith No More,Faith No More,"funk rock, alternative metal, rap metal",63,26,spotify:track:4ReyTz0y3TGkX48wO3Llot
9,Chloe Dancer/Crown Of Thorns - Mother Love Bone,Mother Love Bone,"grunge, post-grunge, funk rock",43,5,spotify:track:7rBMZvgeWnOTHWUh3Pvw51



EXCLUDE ROCK - FIND SIMILAR VIBES IN OTHER GENRES


,track,artist,genres,popularity,playlist_count,track_uri
0,Heaven Is A Halfpipe (If I Die) - OPM,OPM,,41,3,spotify:track:1Sm3U3B5XoidYQEZQZrh3i
1,Jaded - Green Day,Green Day,"punk, pop punk",82,3,spotify:track:6uWp8yAt8dN5ZaT7REJ6RV
2,Stand by Me - Oasis,Oasis,"britpop, madchester",82,6,spotify:track:2gANywSFYF58YFMPdDSAjC
3,Electric Worry - Clutch,Clutch,"stoner rock, stoner metal, sludge metal",54,4,spotify:track:7mQgJx5mfCuI6o8elrYCKg
4,Hitchin' A Ride - Green Day,Green Day,"punk, pop punk",82,5,spotify:track:6WlmA3HPHuNIR2mxI8LkZF
5,Burnout - Green Day,Green Day,"punk, pop punk",82,6,spotify:track:7dVDWf0wud70V4PgYfKnaG
6,Warning - Green Day,Green Day,"punk, pop punk",82,7,spotify:track:0ue0gFLwvXd8qZIPg35Qiu
7,Geek Stink Breath - Green Day,Green Day,"punk, pop punk",82,3,spotify:track:4mI4HF4OVBg3tl2IWzSaoX
8,Long Way Down - The Goo Goo Dolls,The Goo Goo Dolls,,78,3,spotify:track:1J307YEKcn9pH7QT5fXTLt
9,Calm Like a Bomb - Rage Against The Machine,Rage Against The Machine,"rap metal, rap rock, alternative metal",72,11,spotify:track:1hR0fIFK2qRG3f3RF70pb7


### 5. Exclude Known Artists

In [37]:
# Artists you already know well
known_artists = {
    'Drake', 'Kendrick Lamar', 'J. Cole', 'Kanye West', 
    'Future', 'Travis Scott', 'Post Malone', 'Migos'
}

print("=" * 60)
print("DISCOVER NEW HIP-HOP ARTISTS")
print(f"Excluding: {', '.join(known_artists)}")
print("=" * 60)

recommender.recommend(
    drake_uris, n=15,
    exclude_artists=known_artists,
    artist_diversity=1.0,
    popularity=0.3  # Lean toward lesser-known
)

DISCOVER NEW HIP-HOP ARTISTS
Excluding: Kendrick Lamar, Post Malone, Travis Scott, J. Cole, Migos, Kanye West, Future, Drake


,track,artist,genres,popularity,playlist_count,track_uri
0,My Story (Ten Toes Down Challenge) - T the Mes...,T the Messenger,,5,3,spotify:track:49jPkK44gc0y92xc2aP4OC
1,Bounce - Ready Wave,Ready Wave,,0,3,spotify:track:6VzxGogNkfI8PVkazYA2eG
2,For Ya - Bandana Split,Bandana Split,,8,4,spotify:track:3pnwRURls7p0waDfWYKsFx
3,SheDGAF - Broderick Batts,Broderick Batts,,0,4,spotify:track:72tVfRP3l8Oi0Cf6wF1ejm
4,Im Selfish - Silky P,Silky P,,0,3,spotify:track:4qRBCEXmzBodcVDkNE4Cop
5,Spaceships - Cyph,Cyph,,5,3,spotify:track:7GfTwSNozRJQbt5PkyutBx
6,Same Game - Golde,Golde,,4,4,spotify:track:3QctUKTMqWxCMLfx4vY7iI
7,Wicked - Purple Reign,Purple Reign,philly soul,0,4,spotify:track:0bVPZsKQtkBeZCWwN81rT2
8,Pray for Them - DJ Tata,DJ Tata,,0,4,spotify:track:4TxvROgeJ831bsMyiY4Lcw
9,#Forever (My Love) - Silk the Prince,Silk the Prince,,8,5,spotify:track:0HJXFc6A6PbCQPVJ360p3T


## Preset Modes

Convenience functions with pre-configured settings for common use cases.

In [38]:
def radio_mode(recommender, seeds, n=20):
    """Like Spotify radio - safe, popular, similar."""
    return recommender.recommend(
        seeds, n=n,
        popularity=0.7,
        artist_diversity=0.3,
        exploration=0.1
    )

def discovery_mode(recommender, seeds, n=20):
    """Find new artists and hidden gems."""
    return recommender.recommend(
        seeds, n=n,
        popularity=0.2,
        artist_diversity=1.0,
        exploration=0.5
    )

def party_mode(recommender, seeds, n=20):
    """Crowd pleasers - popular hits only."""
    return recommender.recommend(
        seeds, n=n,
        popularity=1.0,
        artist_diversity=0.5,
        exploration=0.2
    )

def deep_dive_mode(recommender, seeds, n=20, genre=None):
    """Explore a specific genre deeply."""
    return recommender.recommend(
        seeds, n=n,
        popularity=0.3,
        artist_diversity=0.8,
        exploration=0.6,
        genres=[genre] if genre else None
    )

def shuffle_mode(recommender, seeds, n=20):
    """Maximum randomness - surprise me!"""
    return recommender.recommend(
        seeds, n=n,
        popularity=0.5,
        artist_diversity=0.7,
        exploration=1.0
    )

In [39]:
print("=" * 60)
print("RADIO MODE - Safe, popular, similar")
print("=" * 60)
display(radio_mode(recommender, drake_uris, n=10))

RADIO MODE - Safe, popular, similar


,track,artist,genres,popularity,playlist_count,track_uri
0,6 Man - Drake,Drake,rap,98,72,spotify:track:4kdfjhj9xNkYU0R8xlDy8k
1,Mariah - NAV,NAV,melodic rap,72,4,spotify:track:2g8nLbse9JTkO78L2apN6M
2,Doubted (feat. Key!) - ILoveMakonnen,ILoveMakonnen,,57,3,spotify:track:4EX8QpApQjdnDeYwAAwfwH
3,For Hot 97 - Lil Yachty,Lil Yachty,,77,7,spotify:track:0qCJuVo2eL3wMBVqL3oTjD
4,Shoot Out The Roof - Lil Yachty,Lil Yachty,,77,12,spotify:track:3V6J0liPAQPjqlyDOvDmKY
5,Outro - Molly Brazy,Molly Brazy,,29,3,spotify:track:4U6FLoVtkwUn8RrDFQb5Ae
6,Bars - Migos,Migos,trap,77,3,spotify:track:3a1JMPNEdjiePcDAZdG82o
7,Litty (feat. Tory Lanez) - Meek Mill,Meek Mill,rap,75,132,spotify:track:5J5PXmMdQ2nh1lZOal8KmK
8,No Role Modelz - Live - J. Cole,J. Cole,rap,85,4,spotify:track:0SFZAQnxM6KBBNi8Y7YflZ
9,Bunny Ranch - Kid Ink,Kid Ink,,69,5,spotify:track:0nNCLhwb0SUFhVMxDY5KLk


In [40]:
print("=" * 60)
print("DISCOVERY MODE - New artists, hidden gems")
print("=" * 60)
display(discovery_mode(recommender, drake_uris, n=10))

DISCOVERY MODE - New artists, hidden gems


,track,artist,genres,popularity,playlist_count,track_uri
0,See Me Fall (Feat. Kensei Abbot) - Ro Ransom,Ro Ransom,,43,11,spotify:track:3zCDpMSxPum6hANAKIskNu
1,Foreva - Cardi B,Cardi B,,82,4,spotify:track:5mwQ8wQElaMO1joDd5vtwd
2,Ambitionz - Meek Mill,Meek Mill,rap,75,3,spotify:track:1mFIiPjGUudqmMDgjH8QOO
3,Bars - Migos,Migos,trap,77,3,spotify:track:3a1JMPNEdjiePcDAZdG82o
4,Highway 85 - Migos,Migos,trap,77,7,spotify:track:2jnn5uL2oWAnG8I9EKvISk
5,Pull Up - Swift,Swift,,19,12,spotify:track:5unrEQAcX0FWaXtIXLTklI
6,• 100 Roses - Jazz Cartier,Jazz Cartier,,42,3,spotify:track:6NbTyTg6wr0R5TRETBPIqg
7,Too Player - Vinny West,Vinny West,hyphy,47,5,spotify:track:0wG4PyMJUxcjvlQnhqfn3x
8,Yacht Master - Swae Lee,Swae Lee,,79,8,spotify:track:589rgFb4eWmV6mFtjSBPhv
9,Hmmm - Hott Headzz,Hott Headzz,,43,17,spotify:track:7bEKga8eFygrhFWB5KTl3A


In [41]:
print("=" * 60)
print("SHUFFLE MODE - Surprise me! (run multiple times)")
print("=" * 60)
display(shuffle_mode(recommender, drake_uris, n=10))

SHUFFLE MODE - Surprise me! (run multiple times)


,track,artist,genres,popularity,playlist_count,track_uri
0,"Drive Fast, Live Young - Tyga",Tyga,,80,5,spotify:track:1xjst9CH1DJJynT8Kxc8RJ
1,Way You Are (feat. Monty) - Fetty Wap,Fetty Wap,,73,6,spotify:track:3NKSqIu5LKDkS5G1rmw0gf
2,Come Down (feat. Chief Keef & Rae Sremmurd) - ...,Mike WiLL Made-It,,66,4,spotify:track:4hQhEDssi8nxX3J8yGg8W7
3,WOA - Migos,Migos,trap,77,6,spotify:track:42yaz95ojl9mpv3PteIYpB
4,March Madness - Future,Future,rap,91,73,spotify:track:26oF6WjkOjDIRK9YsdZp2l
5,Glitta - Tyga,Tyga,,80,4,spotify:track:6eKhUyog7OVjeR8jswqrC4
6,Smoke Dat Weed - Juicy J,Juicy J,crunk,76,3,spotify:track:6VLLvnEXqw7iGIOJwaZpRW
7,Call from the Bank (feat. Mgmt) - Nipsey Hussle,Nipsey Hussle,west coast hip hop,67,3,spotify:track:4aHN7SjWuXXG9fhpxGaME6
8,No Gain - Wiz Khalifa,Wiz Khalifa,rap,84,4,spotify:track:5FAHKmc3kplq3QV0sJxwxF
9,Bounce - Ready Wave,Ready Wave,,0,3,spotify:track:6VzxGogNkfI8PVkazYA2eG


## Interactive Playground

Use the cells below to build your own custom recommendations!

In [ ]:
# Find your own seed tracks - change the search query!
# Only shows tracks that are IN the vocabulary (usable as seeds)
search_query = "dwight yoakam"  # <-- CHANGE THIS TO YOUR ARTIST/SONG

results = recommender.search_tracks(search_query, limit=30)
valid_results = results[results['in_vocab'] == True].sort_values(by='playlist_count', ascending=False)

print(f"Found {len(valid_results)} usable tracks (out of {len(results)} total)")
print("\nCopy track_uri values from below to use as seeds:\n")
valid_results.head(15)

,track,artist,genres,playlist_count,track_pop_pct,artist_popularity,artist_pop_pct,in_vocab,track_uri
1,"Guitars, Cadillacs - Dwight Yoakam",Dwight Yoakam,"classic country, country, honky tonk",31,98,63,69,True,spotify:track:3HtCJiLV8GH87AViMtUKhO
2,A Thousand Miles From Nowhere - Dwight Yoakam,Dwight Yoakam,"classic country, country, honky tonk",19,96,63,69,True,spotify:track:2rjeRpzY5eKn2zZMwAfuod
0,Fast As You - Dwight Yoakam,Dwight Yoakam,"classic country, country, honky tonk",16,95,63,69,True,spotify:track:6FevJcaJvt5EvsHnuJHBnb
6,Honky Tonk Man - Dwight Yoakam,Dwight Yoakam,"classic country, country, honky tonk",11,93,63,69,True,spotify:track:3Hbm5luuxJheCKJheYAVbZ
3,Suspicious Minds - Dwight Yoakam,Dwight Yoakam,"classic country, country, honky tonk",8,91,63,69,True,spotify:track:4kTCYsbseIEd7aJmeCIP2n
12,"Turn It On, Turn It Up, Turn Me Loose - Dwight...",Dwight Yoakam,"classic country, country, honky tonk",6,88,63,69,True,spotify:track:0JyuEt2VTlBvGMBrM1XJ5x
4,Streets Of Bakersfield - Dwight Yoakam,Dwight Yoakam,"classic country, country, honky tonk",5,86,63,69,True,spotify:track:5B7f2auRhwkl5OjC2bQ58f
8,Little Ways - Dwight Yoakam,Dwight Yoakam,"classic country, country, honky tonk",4,82,63,69,True,spotify:track:4XARrCJBmIpZ1gQa5qKz81
11,Little Sister - Dwight Yoakam,Dwight Yoakam,"classic country, country, honky tonk",4,82,63,69,True,spotify:track:4qGsLYHRIXHGlh2l1J0xYo
5,Ain't That Lonely Yet - Dwight Yoakam,Dwight Yoakam,"classic country, country, honky tonk",3,76,63,69,True,spotify:track:0Kwu1D6HJc8KCLjTGCB22w


In [46]:
# Build your own recommendation!
# Paste track URIs from search results above
my_seeds = [
    'spotify:track:3HtCJiLV8GH87AViMtUKhO',
    'spotify:track:2rjeRpzY5eKn2zZMwAfuod',
    'spotify:track:6FevJcaJvt5EvsHnuJHBnb'
]

if my_seeds:
    # Check if seeds are in vocabulary first
    in_vocab = [uri for uri in my_seeds if uri in recommender.vocab]
    print(f"Seeds in vocabulary: {len(in_vocab)}/{len(my_seeds)}")
    
    results = recommender.recommend(
        my_seeds,
        n=15,
        popularity=0.5,         # 0.0=obscure, 1.0=mainstream
        artist_diversity=0.5,   # 0.0=same artists OK, 1.0=new artists only
        exploration=0.3,        # 0.0=deterministic, 1.0=random
        genres=None,            # e.g., ['hip hop', 'rap']
        exclude_genres=None,    # e.g., ['country', 'metal']
        exclude_artists=None    # e.g., {'Drake', 'Kanye West'}
    )
    display(results)
else:
    print("Add some seed track URIs to the my_seeds list above!")

Seeds in vocabulary: 3/3


,track,artist,genres,popularity,playlist_count,track_uri
0,The Dollar - Jamey Johnson,Jamey Johnson,"outlaw country, country, red dirt",61,5,spotify:track:7F48iwtNik16pAg9YTDbUf
1,Ramblin' Fever - Merle Haggard,Merle Haggard,"classic country, outlaw country, country",63,4,spotify:track:7GwnjMIdDZzA1xndx9RRT9
2,Don't Close Your Eyes - Keith Whitley,Keith Whitley,"classic country, country, traditional country",64,24,spotify:track:2SawYQpvsLAEF5lpXWhvyl
3,Walk Softly On This Heart Of Mine - The Kentuc...,The Kentucky Headhunters,"southern rock, classic country, country rock",46,3,spotify:track:5yma4EpKYJ4r6jXhebUzh2
4,Adalida - George Strait,George Strait,"country, classic country, traditional country",76,5,spotify:track:7qQLecGsf4gHLH28lyaU02
5,Coward Of The County - Kenny Rogers,Kenny Rogers,"country, classic country",73,5,spotify:track:2UpYSZeFdpfnXL0NCEqQnP
6,Sangria Wine - Jerry Jeff Walker,Jerry Jeff Walker,"texas country, outlaw country, red dirt",44,6,spotify:track:7xjQ0YNPtC8R9tEdFNOFcG
7,Where The Stars And Stripes And The Eagle Fly ...,Aaron Tippin,"classic country, country, traditional country",51,8,spotify:track:1tTZM7LUDkxcykOVVIGW4W
8,Take Me Down - Alabama,Alabama,"classic country, country, country rock",68,5,spotify:track:7HFkkFUwbGiQd6Hz2uS9kZ
9,One More Last Chance - Vince Gill,Vince Gill,"classic country, country, traditional country",64,5,spotify:track:2h5a6t9JFasqH5WJFPbmGL


## Summary

The `ControllableRecommender` gives you fine-grained control over recommendations:

| Parameter | Low (0.0) | High (1.0) |
|-----------|-----------|------------|
| `popularity` | Underground/obscure | Mainstream hits |
| `artist_diversity` | Same artists OK | New artists only |
| `exploration` | Deterministic/safe | Random/adventurous |

### Filtering Options
- `genres` - Include only tracks matching these genres
- `exclude_genres` - Filter out tracks with these genres
- `exclude_artists` - Skip artists you already know

### Preset Modes
- **Radio Mode**: Safe, popular, similar (like Spotify radio)
- **Discovery Mode**: Find new artists and hidden gems
- **Party Mode**: Crowd pleasers and popular hits
- **Deep Dive Mode**: Explore a genre deeply
- **Shuffle Mode**: Maximum randomness - surprise me!